# Week 1: RNN

### (video 1) Sequence problems
Examples of sequence data
- speach recognition: x(audio = sequence) => y(transcript = sequence)
- music generation: x(nan; single integer, referring to a style) => y(sequence) (T<sub>x</sub> ≠ T<sub>y</sub>)
- sentiment classification: x(prase = sequence) => y(stars) (T<sub>x</sub> ≠ T<sub>y</sub>)
- DNA sequence analysis: x(DNA code = sequence) => y (which part corresponds to a protein = sequence)
- machine translation: x(phrase = sequence) => y(translation = sequence) (T<sub>x</sub> ≠ T<sub>y</sub>)
- video activity recognition: x(videoframes = sequence) => y(activity)
- NER: x(phrase = sequence) => y(entities)

Problems can be addressed as supervised learning with labled data X, and Y as a training set
=> but different types of sequence problems
- X and Y are sequences
- X and Y can have different length or same 
- only X or Y can be the sequence

### (video 2) Notations to define sequence problems
NER: identify people


|  | word | word | word | word | word | word | word | word | word |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| input x: | Harry | Potter | and | Hermione | Granger | invented | a | new | spell |
| input features: | x<sup><1></sup> | x<sup><2></sup>  | ... | ... | x<sup>&lt;t&gt;</sup>  | ... | ... | x<sup><8></sup>  | x<sup><9></sup>  |
| output y1: | 1 | 1 | 0 | 1 | 1 | 0 | 0 | 0 | 0 |
| output: | y<sup><1></sup> | y<sup><2></sup>  | ... | ... | y<sup>&lt;t&gt;</sup>  | ... | ... | y<sup><8></sup> | y<sup><9></sup>  |

y1 – not the best representation, since doesn't tell you, where is the start and end of peoples' names

the input is 9 words => we'll have 9 sets of features representing these
- x<sup>&lt;t&gt;</sup> – feature in the middle of the sequence
- *t* implies that these are temporal sequences (but it will be used regardless the type of sequence)
- T<sub>x</sub> – length of the input sequence = 9
- T<sub>y</sub> – length of the output sequence = 9
- T<sub>x</sub> and T<sub>y</sub> can be different
        
- X<sup>(i)</sup> – i-th training example, in this case, this particular phrase
- x<sup>(i)&lt;t&gt; </sup> – t-th element of an input sequence of i-th training example   
- T<sub>x</sub><sup>(i)</sup> – the length of an input sequence of i-th training example  
- y<sup>(i)&lt;t&gt; </sup> – t-th element of an output sequence of i-th training example 
- T<sub>y</sub><sup>(i)</sup> – the length of an output sequence of i-th training example
    
**Representation of individual words of the sentence**
- You come up with a vocabulary = dictionary
- Usually 30-50k words, around 100k – not uncommon as well
    
| words | number |
| --- | --- |
|a| 1 |
|aaron| 2 |
|...| ... |
|and| 367 |
|...| ... |
|harry| 4075 |
|...| ... |
|potter| 6830 |
|...| ... |
|zulu| 10000 |

- to build this dictionaly: one of ways: find top 10k occuring words
- then use one hot representations to encode each of this words:
    - x<sup><1></sup> = vetor [0, ... , 1, ...., 0] with unique 1 in 4075-th position

And the *goal* is given this representation for X to learn a mapping using a sequence model to then target output y, I will do this as a supervised learning problem, I'm sure given the table data with both x and y.
    
If you encounter a word that is not in your vocabulary, you create a new token or a new fake word called *Unknown* = UNK

### (video 3) Recurrent Neural Network (RNN) model

**Why not standard network?**

9 input words = [x<sup><1></sup>, ..., x<sup>&lt;t&gt;</sup>, ..., x<sup>&lt;T<sub>x</sub>&gt;</sup>]

feed them into standard NN of a few layers => 
output of 9 values 0 or 1 = [y<sup><1></sup>, ..., y<sup>&lt;t&gt;</sup>, ..., y<sup>&lt;T<sub>y</sub>&gt;</sup>] 
    
which gonna tell you whether each of those words are part of persons name

But this not gonna work, cause:
- Inputs and outputs can be of different lengths in different examples (and even no maximum sentence length is known)
- Doesn't share feautures learnt across different positions of text: "Harry" in first position recognized as a name won't imply that "Harry" in some other position will be recognized as such

Similarly to CNN 
- you want things learned for one part of the image to generalize quickly to other parts of the input (in CNN - image, in SM - sequence)
- better representation will also let you reduce the number of parameters in your model

If we think of reqular network: each of x<sup>&lt;t&gt;</sup> is a 10k onehot encoded vector => so the total input size would be 10k x max amount of words in phrase => the weight matrix would be too huge

**What is RNN?**
- We read sentence from left to right
- We take first word x<sup>&lt;1&gt;</sup> and feed it to a first NN layer. We try to predict an output y_hat<sup>&lt;1&gt;</sup>: whether it is a part of person's name or not
- RNN when it goes to read second word x<sup>&lt;2&gt;</sup>, 
    - instead of just predicting y_hat<sup>&lt;2&gt;</sup> using x<sup>&lt;2&gt;</sup>, 
    - it also gets some information from the comptuation of step 1
    - in particular: an **activation** value from time step one is passed to time step 2: a<sup>&lt;1&gt;</sup>
- and so on for each next word: input x<sup>&lt;t&gt;</sup> => y_hat<sup>&lt;t&gt;</sup>
- till the last ont: input x<sup>&lt;T<sub>x</sub>&gt;</sup> => y_hat<sup>&lt;T<sub>y</sub>&gt;</sup>
- In this example, T<sub>x</sub> = T<sub>y</sub>, but the architecture will change a bit if T<sub>x</sub> ≠ T<sub>y</sub>
- At the begining we pass an activation a<sup>&lt;0&gt;</sup> which can be 
    - randomly initiated
    - vector of zeros
    
Representation of RNN: 

x<sup>&lt;t&gt;</sup> => [ooooooooo] (loop + shaded box = time delay of one step) => y<sup>&lt;t&gt;</sup>

**Parameters**
- The recurrent neural network scans through the data from left to right. 
- The parameters it uses for each time step are shared  
    - w<sub>ax</sub> = govern the connection from X1 to the hidden layer
        - The second index means that this w<sub>ax</sub> is going to be multiplied by some X-like quantity
        - "a" means that this is used to compute some a-like quantity (see equations below)
    - w<sub>aa</sub> =  govern the horizontal connections = the activations
    - w<sub>ya</sub> = govern the output predictions

So, to make a prediction for y_hat<sup>&lt;3&gt;</sup>, this RNN gets the information not only from x<sup>&lt;3&gt;</sup> but also the information from x<sup>&lt;1&gt;</sup> and x<sup>&lt;2&gt;</sup> because the information on x<sup>&lt;1&gt;</sup> can pass horizontally

one **weakness** of this RNN is that it only uses the information that is earlier in the sequence to make a prediction. This can be overcome in Bidirectional RNN

**What are the calculations this RNN does?**
- a<sup>&lt;0&gt;</sup> = [0] (vector)
- a<sup>&lt;1&gt;</sup> = g(w<sub>aa</sub> * a<sup>&lt;0&gt;</sup> + w<sub>ax</sub> * x<sup>&lt;1&gt;</sup> + b<sub>a</sub>)
    - g = activation function, 
        - often is "tanh" 
        - RelU (preventing the vanishing gradient problem)
    - b<sub>a</sub> = bias    
- y_hat<sup>&lt;1&gt;</sup> = f(w<sub>ya</sub> * a<sup>&lt;1&gt;</sup> + b<sub>y</sub>)
    - f = activation function (same or another), depends on what the output is
        - binary classification problem (NER) = sigmoid activation function
        - k-way classification problem = softmax
    - b<sub>y</sub> = bias
    
More generaly 
- a<sup>&lt;t&gt;</sup> = g(w<sub>aa</sub> * a<sup>&lt;t-1&gt;</sup> + w<sub>ax</sub> * x<sup>&lt;t&gt;</sup> + b<sub>a</sub>)
- y_hat<sup>&lt;t&gt;</sup> = f(w<sub>ya</sub> * a<sup>&lt;t&gt;</sup> + b<sub>y</sub>)

Simplification of these two equations
- a<sup>&lt;t&gt;</sup> = g (w<sub>a</sub> * [a<sup>&lt;t-1&gt;</sup>, x<sup>&lt;t&gt;</sup>]+ b<sub>a</sub>)
    - w<sub>a</sub> is a matrix which is defined as a horizontal stack of matricies [w<sub>aa</sub>; w<sub>ax</sub>]
    - if dim(a) = 100 and dim(x) =10k => 
        - dim(w<sub>aa</sub>) = 100; 100
        - dim(w<sub>ax</sub>) = 100; 10k
        - dim(w<sub>a</sub>) = 100; 10100
    - [a<sup>&lt;t-1&gt;</sup>, x<sup>&lt;t&gt;</sup>] is two vectors stacked together vertically 
        - dim(a<sup>&lt;t-1&gt;</sup>) = 100, 1
        - dim(x<sup>&lt;t&gt;</sup>) = 10k, 1
        - dim([a<sup>&lt;t-1&gt;</sup>, x<sup>&lt;t&gt;</sup>]) = 10100, 1
    - The advantage of this notation is that rather than carrying around two parameter matrices, Waa and Wax, we can compress them into just one parameter matrix Wa
- y_hat<sup>&lt;t&gt;</sup> = f(w<sub>y</sub> * a<sup>&lt;t&gt;</sup> + b<sub>y</sub>) (similarly)
    - w<sub>y</sub> and b<sub>y</sub> denotes what type of output quantity we're computing.
        - w<sub>y</sub> indicates a weight matrix or computing a y-like quantity, 
        - w<sub>a</sub> and b<sub>a</sub>  on top indicates that thes eparameters are for computing activation output quantity

### (video 4) Back propagation through time

**Element-wise (individual time steps) loss function**

A certain word in the sequence is supposed to be a person's name: y<sup>&lt;t&gt;</sup> = 1
And the NN outputs some probability of the particular word being a person's name: y_hat<sup>&lt;t&gt;</sup> = 0.1
Loss = standard logistic regression loss, also called the cross entropy loss

L<sup>&lt;t&gt;</sup>(y_hat<sup>&lt;t&gt;</sup>, y<sup>&lt;t&gt;</sup>) = - y<sup>&lt;t&gt;</sup> * log(y_hat<sup>&lt;t&gt;</sup>) - (1-y<sup>&lt;t&gt;</sup>) * log(1-y_hat<sup>&lt;t&gt;</sup>)

**Overall loss**

L(y_hat, y) = sum(t=1>T<sub>y</sub>) [L<sup>&lt;t&gt;</sup>(y_hat<sup>&lt;t&gt;</sup>, y<sup>&lt;t&gt;</sup>)]

Backprop requires doing computations in the opposite directions. And that then, allows you to compute all the appropriate quantities that lets you then, take derivatives, respected parameters, and update the parameters using gradient descent

In back propagation procedure, the most significant message (or the most significant recursive calculation) is the one between activations. And it gives this algorithm the name "backpropagation through time".


### (video 5) Different Types of RNNs

**Example 1 (previous): many-to-many architecture with T<sub>x</sub> = T<sub>y</sub>**

<img src='./Images/0004.png'></img>

**Example 2: sentiment classification: many-to-one **

* x = text = "There is nothing to like in this movie.
* y = 0/1 (positive or negative review) or [1, 5]

Simplification of the architecture: rather than having to use an output at every single time-step, we can then just have the RNN read into entire sentence and have it output y at the last time-step when it has already input the entire sentence

<img src='./Images/0003.png'></img>

**Example 3: standard NN covered in first two courses: one-to-one**

<img src='./Images/0001.png'></img>

**Example 4: music generation: one-to-many**

* input = x = null or any integer defining genre or smth else
* output = set of notes of the musical piece. 
* If you want, you can have this input a(0) as well
* technically when you're actually generating sequences, often you take these first synthesized output and feed it to the next layer as well, and so on

<img src='./Images/0002.png'></img>

**Example 5: machine translation: many-to-many architecture with T<sub>x</sub> ≠ T<sub>y</sub>**

2 distinct parts of the RNN: 
* the encoder 
* the decoder

<img src='./Images/0005.png'></img>

**Example 6: will be shown on week 4**

Technically, there's one other architecture – attention based architectures. It isn't cleanly captured by one of the diagrams we've drawn so far.

**NB** there are some subtleties with sequence generation, which is what we'll discuss in the next video

### (video 6) Language Model and Sequence Generation

Language modeling is one of the most basic and important task in natural language processing. It is also one that RNNs do very well. In this video, you'll learn about how to build a language model using an RNN

**What is a language model?**

* the apple and pear salad was delicious. 
    * What did you just hear me say? 
        * Did I say the apple and pair salad? 
        * Or did I say the apple and pear salad?
* Speech recognition system picks the second sentence by using language model which tells it what is the probability of either of these two sentences.
* language model, given any sentence, tells you what is the probability of that particular sentence
    * probability of sentence = if you were to pick up a random newspaper, open a random email, or pick a random webpage, or listen to the next thing someone says, what is the chance that the next sentence you read somewhere out there in the world will be a particular sentence?
* fundamental component for both 
    * speech recognition systems 
    * for machine translation systems
    
<img src='./Images/0006.png'></img>
    
P(sentence) = P(y<sup>&lt;1&gt;</sup>, y<sup>&lt;2&gt;</sup>, ..., y<sup>&lt;T<sub>y</sub>&gt;</sup>)

= probability of that particular sequence of words

**How do you build a language model?**

* (1) Training set: large corpus of text in a certain language
* Let's say you get a sentence in your training set as follows: "cats average 15 hours of sleep a day""
* (2) You tokenize it: form a vocab and encode each word with one hot vectors
    * You model when the sentence ends: form an exra token: < EOS > token
        * It's not necessary 
        * Period can be a token as well
    * If there is a word in the sentence abscent in the vocab = < UNK > token
        * and we just model the chance of the unknown word instead of the specific word
* (3) build an RNN to model the chance of these different sequences
    * (input) x<sup>&lt;t&gt;</sup> = y<sup>&lt;t-1&gt;</sup> (output)
    * At time zero, you're going to end up computing some activation a<sup>&lt;1&gt;</sup> as a function of some input x<sup>&lt;1&gt;</sup>
    * and x<sup>&lt;1&gt;</sup> would just be set to zero vector. 
    * The previous a<sup>&lt;1&gt;</sup> by convention, also set that to vector zeros. 
    * But what a<sup>&lt;1&gt;</sup> does is it will make a Softmax prediction to try to figure out what is the probability of the first word y, so that's going to be y_hat<sup>&lt;1&gt;</sup>. 
        * What this step does is really it has a Softmax, 
        * so it's trying to predict what is the probability of any word in a dictionary, 
        * what's the chance that the first word is a, what's the chance that the first word is Aaron, 
        * what's the chance that the first word is cats, 
        * what's the chance the first word is Zulu,
        * what's the chance that the first word is an unknown word, 
    * y_hat<sup>&lt;1&gt;</sup> is output according to a Softmax, it just predicts what's the chance that the first word being "cats"
    * At the second step, the output is again predicted by Softmax, the RNN's job is to predict what's the chance of it being whatever word it is, given what had come previously: cats
    
<img src='./Images/0007.png'></img>

* (4) Define the cost function
    *  At a certain time t, if the true word was y<sup>&lt;t&gt;</sup> and your network Softmax predicted some y_hat<sup>&lt;t&gt;</sup>, then this is the Softmax loss function 
    * and then the overall loss is just the sum over all time steps of the losses associated with the individual predictions.
    * Given the sentence "y<sup>&lt;1&gt;</sup> y<sup>&lt;2&gt;</sup> y<sup>&lt;3&gt;</sup>)"
    * The first softmax tells the probability of the word y<sup>&lt;1&gt;</sup>
    * And we end up with P(y<sup>&lt;1&gt;</sup>, y<sup>&lt;2&gt;</sup>, y<sup>&lt;3&gt;</sup>) = P(y<sup>&lt;1&gt;</sup>) * P (y<sup>&lt;2&gt;</sup>| y<sup>&lt;1&gt;</sup>) * P (y<sup>&lt;3&gt;</sup>| y<sup>&lt;1&gt;</sup>, y<sup>&lt;2&gt;</sup>)
    
<img src='./Images/0008.png'></img>

### (video 7) Sampling Novel Sequences

After you train a sequence model, one of the ways you can informally get a sense of what is learned is to have a sample novel sequences

Sequence model, models the chance of any particular sequence of words. We like it to do is sample from this distribution to generate noble sequences of words.

<img src='./Images/0009.png'></img>

The network was trained using this structure shown at the top. But to sample, you do something slightly different
* you input the usual 
    * x<sup>&lt;1&gt;</sup> = 0, 
    * a<sup>&lt;1&gt;</sup> = 0. 
* And now your first time stamp will have softmaxmax probability over possible outputs (vector of word probabilities)
    * And then you take this vector and use, for example, the numpy command np.random.choice to sample the first word according to distribution defined by this vector probabilities.
* The second time step is expecting this y_hat<sup>&lt;1&gt;</sup> as input.
    * The soft max will make a prediction for what is y_hat<sup>&lt;2&gt;</sup> 
    * The neuron will give you the vector of probabilities for all vocab words given the chosen first word 
    * And you sample from most probable words of this vector
* If the end of sentence token is part of your vocabulary, you could keep sampling until you generate an EOS token
    *  alternatively, you can just decide to sample 20 words and keep going until you've reached that number of time steps
*  this particular procedure will sometimes generate an unknown word token. If you want to make sure that your algorithm never generates tit, you can reject any sample that came out as unknown word token and just keep resampling from the rest of the vocabulary until you get a word that's not an unknown word.


Depending on your application, one thing you can do is also build a **character level RNN**

Vocab = [a, ... , z, , . , ; , 0, ... ,9, A, ....]

Your sequence y<sup>&lt;1&gt;</sup>, y<sup>&lt;2&gt;</sup>, y<sup>&lt;3&gt;</sup> would be the individual characters in your training data, rather than the individual words in your training data

Using a character level language model has some pros and cons. 
* Pros
    * you don't ever have to worry about unknown word tokens. 
        * the model is able to assign a sequence like mau, a non-zero probability. 
        * Whereas if mau was not in your vocabulary for the word level language model, you just have to assign it the unknown word token. 
* Cons
    * you end up with much longer sequences: many english sentences will have 10 to 20 words but may have many, many dozens of characters. 
    * Athe model is not as good as word level language models at capturing long range dependencies between how the the earlier parts of the sentence also affect the later part of the sentence. 
    * And character level models are also just more computationally expensive to train. 

So the trend in NLP is that for the most part, word level language model are still used, but as computers gets faster there are more and more applications where people are, at least in some special cases, starting to look at more character level models.

**Further we'll discuss challenges of training RNNs and how to deal with them**

### (video 8) Vanishing Gradients problem with RNNs

* The **cat**, which already ate ...., **was** full. 
* The **cats**, which already ate ..., **were** full.

in English this stuff in the middle (...) could be arbitrarily long.

The basic RNN we've seen so far is not very good at capturing very long-term dependencies
* The same problem of vanishing gradients appears for very deep neural networks. 
    * You would carry out forward prop from left to right and then backprop. 
    * If this is a very deep neural network, then the gradient from this output y would have a very hard time propagating back to affect the weights of these earlier layers, to affect the computations of the earlier layers.
* RNN has similar problem 
    * It can be quite difficult for the outputs of the errors associated with the later timesteps to affect the computations that are earlier. 
    * In practice, what this means is it might be difficult to get a neural network to realize that it needs to memorize.
    
In basic RNN model has many local influences: the output y_hat<sup>&lt;3&gt;</sup> is mainly influenced by values close to y_hat<sup>&lt;3&gt;</sup>
    
**For very Deep NN we also talked about exploding gradients**

* While doing backprop, the ingredients should not just decrease exponentially they may also increase exponentially with the number of layers you go through. 
    * It turns out that vanishing gradients tends to be the biggest problem with training RNNs. 
    * Although when exploding gradients happens it can be catastrophic because the exponentially large gradients can cause your parameters to become so large that your neural network parameters get really messed up.
    
* It turns out that exploding gradients are easier to spot because the parameter has just blow up. 
    * You might often see NaNs, not a numbers, meaning results of a numerical overflow in your neural network computation. 
    * If you do see exploding gradients, one solution to that is apply **gradients clipping**
        * look at your gradient vectors, 
        * and if it is bigger than some threshold, 
        * re-scale some of your gradient vectors so that it's not too big, 
        * so that is clipped according to some maximum value.
* vanishing gradients is much harder to solve, which we are going to discuss in the next sections 

### (video 9) Gated Recurrent Unit (GRU)

* GRU is a modification to the RNN hidden layer that makes it much better at capturing long-range connections and helps a lot with the vanishing gradient problem

You've already seen the formula for computing the activations at time t of an RNN. 
* It's the activation function applied to 
    * the parameter W_a 
        * times the activations for a previous time sediment the current input 
    * plus the bias. 

<img src='./Images/0010.png'></img>

we're going to use a similar picture to explain the GRU

* The cat, which already ate ...., was full.
* The cats, which already ate ..., were full.


**Simplified GRU**

the GRU unit is going to have a **new variable called C**, which stands for cell, for memory cell. 
* What the memory cell do is it will provide a bit of memory. 
* Remember, for example, whether cat was singular or plural, so that when it gets much further into the sentence, it can still work on the consideration whether the subject of the sentence was singular or plural.
* At time t, the memory cell will have some value **с<sup>&lt;t&gt;</sup>**
* the GRU unit will actually output an activation value  **a<sup>&lt;t&gt;</sup>** =  **с<sup>&lt;t&gt;</sup>**
    * different symbols c and a are used to denote the memory cell value and the output activation value even though they're the same 
    * I'm using this notation because when we talked about LSTMs a little bit later, these will be two different values.
    
These are the equations that govern the computations of a GRU unit. 
* At every time step, we're going to consider an overwriting the memory cell with a value c tilde of t. 
* This going to be a **candidate** for replacing c of t. 
* We're going to compute this using an activation function, tanh 
    * of w_c, and so that's the parameter matrix w_c and we'll pass it as parameter matrix.
    
<img src='./Images/0011.png'></img>    

The important idea of the GRU, it will be that we'll have a **gate**. 
* The gate I'm going to call Gamma_u = a value belonging to [0; 1] = sigmoid function
    * u stands for update gate
    * Gamma is very close to 0 or to 1 most of the time
* Then next the key part of the GRU is the equation
    * we have come up with a candidate where we're thinking of updating C using c tilde 
    * and then the gate will decide whether or not we actually update it
        * this memory cell C is going to be set to either zero or one depending on whether the word you're conserving, really the subject of the sentence is singular or plural
        *  the GRU unit will memorize the value of the C^t all the way down, where this is still equal to one and so that tells it was singular so use the choice was. 
        * The job of the gate, of gamma u, is to decide when do you update this value.
        * when you see the phrase the cat, you know that you're talking about a new concept, the subject of the sentence cat. That would be a good time to update this bit and then maybe when you're done using it the cat was full, then you know I don't need to memorize anymore I can just forget that.

* The specific equation we'll use for the GRU is the following:

<img src='./Images/0012.png'></img>   

* if the gate is equal to one, then c<sup>&lt;t&gt;</sup> equal to this candidate value so that's like over here, set the gate equal to one so go ahead and update that bit
* Then for all of these values in the middle, you should have the gate equal zero so do the same, don't update it

<img src='./Images/0013.png'></img>   

Simplified version of GRU unit (violet box is the equation for c<sup>&lt;t&gt;</sup> from the picture above):
* the value of the memory cell from the previous step is taken along with an input
* tanh is applied to calculate the candidate for the next value of a memory cell
* sigmoid is applied to calculate the gate value, defining whether the candidate would be accepted
* all of them are used to calculate 
    * the next value of memory cell 
    * as well as an output (after the application of softmax function)

<img src='./Images/0014.png'></img>   

* because the gate is quite easy to set to zero so long as sigmoid argument is a large negative value, 
    * then up to numerical round-off, the update gate will be essentially zero, very close to zero. 
    * When that's the case, then this update equation and sub setting c<sup>&lt;t&gt;</sup> = c<sup>&lt;t-1&gt;</sup> 
    * so this is very good at maintaining the value for the cell 
    * (!) and because gamma can be so close to zero, can be 0.000001 or even smaller than that, **it doesn't suffer from much of a vanishing gradient problem**
    
**Implementation**
* In the equations have written, c<sup>&lt;t&gt;</sup> can be a vector (100 dim)
* c_tilda<sup>&lt;t&gt;</sup> also 100 dim
* Gamma also the same dimention of 100
* So the multiplications in the  c<sup>&lt;t&gt;</sup> equation are element-wise multiplications
* If Gamma = 100-dimensional vector of bits, the value is mostly 0 and 1, that tells you of this **100-dimensional memory cell**, which are the bits you want to update.
    * these element-wise multiplications do is it just tells you GRU **which are the dimensions of your memory cell vector to update** at every time step. 
    * You can choose to keep some bits constant while updating other bits. 
    * For example, 
        * maybe you'll use one-bit to remember the singular or plural cat, 
        * and maybe you'll use some other bits to realize that you're talking about food. 
        * Because we talked about eating and talk about foods, then you'd expect to talk about whether the cat is full later
        
**Full GRU**
* We add one more gate = Gamma_r
    * r stands for relevance
    * This gate Gamma r tells you how relevant is c<sup>&lt;t-1&gt;</sup> to computing the next candidate for c<sup>&lt;t&gt;</sup>

<img src='./Images/0015.png'></img>    

Why not use a simpler version from the previous section? Experiments show that it works better. Other common version is called an LSTM, which stands for Long, Short-term Memory

*Notations: sometimes h_tilda, u, r, h are used in academic literature*

### (video 10) Long Short Term Memory (LSTM)

More powerful and general than GRU

<img src='./Images/0016.png'></img>

* For the LSTM, we will no longer have the case that a&lt;t&gt; is equal to c&lt;t&gt;. 
* So this is what we use and so this is like the equation on the left except that 
    * we now more explicitly use a<sup>&lt;t-1&gt;</sup> not c<sup>&lt;t-1&gt;</sup>; 
    * and we are not using this gamma r this relevance
* Update gate the same as before
* But instead of having one update gate control both of these terms, we're going to have two separate terms. Forget gate Gamma_f. 
    * this gives the memory cell the option of keeping the old value c&lt;t-1&gt; and then just adding to it this new value c_tilda<sup>&lt;t&gt;</sup>.
     * So use a separate update and forget gates right? 
* And new output gate

**Variations**

* **peephole connection** – the most common one
    * instead of just having the gate values be dependent only on a<sup>&lt;t-1&gt;</sup>, x<sup>&lt;t&gt;</sup>, 
    * Sometimes people also sneak in there the value c<sup>&lt;t-1&gt;</sup>  as well
    * these are vectors, so each X-th element of c<sup>&lt;t-1&gt;</sup> affects corresponding X-th elements of each gate 

<img src='./Images/0017.png'></img>

**when should you use a GRU and when should you use an LSTM?** 
* There is a widespread consensus in this. 
* LSTMs actually came much earlier and then GRUs were relatively recent invention that were maybe derived as partly a simplification of the more complicated LSTM model. 
* Researchers have tried both of these models on many different problems and on different problems the different algorithms will win out. 
    * the advantage of the GRU is that it's a simpler model. And so it's actually easier to build a much bigger network only has two gates, so computation runs a bit faster so it scales the building, somewhat bigger models. 
    * But the LSTM is more powerful and more flexible since there's three gates instead of two. 
* If you want to pick one to use, I think LSTM has been the historically more proven choice. Although I think the last few years GRUs have been gaining a lot of momentum: it might be easier to scale them to even bigger problems.

### (video 11) Bidirectional RNN

<img src='./Images/0018.png'></img>

these cells are standard RNN blocks, or GRU units, or LSTM blocks

* So we have four inputs, X1 through X4. 
* So this network's hidden there will have a forward recurring components. So I'm going to call this a1, a2, a3, and a4. 
* And so each of these four recurrent units influence the current X.
* To help predict y hat 1, y hat 2, y hat 3, and y hat 4
* Plus backward recurrent component

<img src='./Images/0019.png'></img>

* This network defines an acyclic graph
    * And so, given an input sequence X1 to X4, the four sequence we first compute a for (1), then use that to compute a for (2), then a for (3), then a for (4). 
    * Whereas the backward sequence will start by computing a backward four and then go back and compute a backward three. 
    * And notice your computing network activations. This is **not back prop!**, this is forward prop
    * When you compute all forward and backward activations, you finally can get your predictions: taking into account past and future
    
if you have an NLP problem, and you have a complete sentence, you're trying to label things in the sentence, a bidirectional RNN with LSTM blocks would be a pretty reasonable first thing to try
    
**Disadvantage of BRNN**

You do need the entire sequence of data before you can make predictions anywhere (speech recognition system won't work on straightforward application of BRNN)

### (video 12) Deep RNNs

The different versions of RNNs you've seen so far will already work quite well by themselves. But for learning very complex functions sometimes is useful to stack multiple layers of RNNs together to build even deeper versions of these models

**standard neural network** 
* you will have an input X.
* And then that's stacked to some hidden layer that might have activations a1 for the first hidden layer, 
* And then that's stacked to the next layer with activations a2, 
* then maybe another layer, activations a3 
* and then you make a prediction ŷ. 

<img src='./Images/0020.png'></img>


**deep RNNs**
* a<sup>[ l ] &lt;t&gt;</sup>
    * l = layer
    * t = time
* How a particular activation is computed: the same parameters Wa[l] and ba[l] are used for every one of these computations at the layer "l". 

<img src='./Images/0021.png'></img>

* whereas for standard NNs we've seen neural networks that are very, very deep, maybe over 100 layers. 
* For RNNs, having three layers is already quite a lot. 
    * Because of the temporal dimension, these networks can already get quite big even if you have just a small handful of layers. 
    * And you don't usually see these stacked up to be like 100 layers.
    * One thing you do see sometimes is that you have recurrent layers that are stacked on top of each other 
        * Plus then you might take an output and then just have a bunch of deep layers (on top of it) that are not connected horizontally but have a deep network, that then finally predicts y<1>.
* blocks can be standard RNN blocks, or GRU units, or LSTM blocks

**NB** deep RNNs are quite computationally expensive to train

# Quiz

<img src='./Images/Q1_1.png'></img>
<img src='./Images/Q1_2.png'></img>
<img src='./Images/Q1_3.png'></img>
<img src='./Images/Q1_4.png'></img>
<img src='./Images/Q1_5.png'></img>
<img src='./Images/Q1_6.png'></img>
<img src='./Images/Q1_7.png'></img>
<img src='./Images/Q1_8.png'></img>